<a href="https://colab.research.google.com/github/anirudhc5/ACLHacks2024/blob/main/AcademiesHacks2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import pandas as pd

In [ ]:
df = pd.read_csv("/content/data.csv")
df.head()

,tweet_text,cyberbullying_type
0,rape is real..zvasiyana nema jokes about being...,gender
1,You never saw any celebrity say anything like ...,gender
2,"@ManhattaKnight I mean he's gay, but he uses g...",gender
3,RT @Raul_Novoa16: @AliciaBernardez @Alex_Aim @...,gender
4,Rape is rape. And the fact that I read one pos...,gender


In [ ]:
df["classification"] = df["cyberbullying_type"].replace({"not_cyberbullying":0, "cyberbullying":1, "other_cyberbullying":1, "age":1, "ethnicity":1, "gender":1, "religion":1,})
df = df.drop("cyberbullying_type", axis=1)
df.head()

,tweet_text,classification
0,rape is real..zvasiyana nema jokes about being...,1
1,You never saw any celebrity say anything like ...,1
2,"@ManhattaKnight I mean he's gay, but he uses g...",1
3,RT @Raul_Novoa16: @AliciaBernardez @Alex_Aim @...,1
4,Rape is rape. And the fact that I read one pos...,1


In [ ]:
texts = df["tweet_text"].tolist()
labels = df["classification"].tolist()

In [ ]:
!pip install huggingface_hub

In [ ]:

# Load model and tokenizer
model_name = "openai-community/roberta-base-openai-detector"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)


In [ ]:
def map_example_to_dict(input_ids, attention_masks, labels):
    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
    }, labels

# Prepare dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    encodings['input_ids'],
    encodings['attention_mask'],
    labels
))
train_dataset = train_dataset.map(map_example_to_dict).shuffle(1000).batch(16)



In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
from transformers import create_optimizer

In [ ]:
from tensorflow import keras

In [ ]:
optimizer, lr_scheduler = create_optimizer(
    init_lr = 5e-5,
    num_train_steps = len(train_dataset) * 5,
    num_warmup_steps=0
)

In [ ]:
model.compile(
    optimizer=optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy("accuracy")]
)

In [ ]:
model.fit(train_dataset, epochs=5, verbose=1)

In [ ]:
model.save_pretrained("/content/model")
tokenizer.save_pretrained("/content/model")

In [ ]:
!zip -r /content/model.zip /content/model

  adding: content/model/ (stored 0%)
  adding: content/model/vocab.json (deflated 59%)
  adding: content/model/merges.txt (deflated 53%)
  adding: content/model/tokenizer_config.json (deflated 76%)
  adding: content/model/tf_model.h5 (deflated 7%)
  adding: content/model/special_tokens_map.json (deflated 52%)
  adding: content/model/config.json (deflated 50%)
  adding: content/model/tokenizer.json (deflated 72%)
